# Convert Datasets

In [ ]:
import warnings

warnings.filterwarnings('ignore')

from nilmtk.dataset_converters.redd import convert_redd

filename = "/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/redd.hdf5"

convert_redd.convert_redd("/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/REDD/low_freq", filename)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

from nilmtk.dataset_converters.greend import convert_greend

filename = "/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/GREEND/greend.hdf5"

convert_greend.convert_greend("/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/GREEND", filename)

In [ ]:
import warnings

warnings.filterwarnings('ignore')

from nilmtk.dataset_converters.ukdale import convert_ukdale

filename = "/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/ukdale/ukdale.hdf5"

convert_ukdale.convert_ukdale("/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/ukdale", filename)

# Prepare Training Data

In [ ]:
from nilmtk import DataSet

#Select redd training data (building 1)
train_redd = DataSet("/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/REDD/redd.hdf5")
train_redd.set_window(end="30-4-2011") #Use data only until 4/30/2011
train_redd_elec = train_redd.buildings[3].elec

#Select greend training data
train_greend = DataSet("/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/GREEND/greend.hdf5")
train_greend.set_window(end="30-4-2014") #Use data only until 4/30/2011
train_greend_elec = train_greend.buildings[3].elec

# Select redd training data (building 1)
# train = DataSet("/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/REDD/redd.hdf5")
# train.set_window(start="10-5-2011") #Use data only until 4/30/2011
# train_elec = train.buildings[1].elec


In [ ]:
from grudisaggregator import GRUDisaggregator
gru = GRUDisaggregator()

In [ ]:
# print(train_greend_elec)
# print(train_redd_elec)
print(train_elec)

# Train

In [ ]:
import pandas as pd
train_redd_mains = train_redd_elec.mains().all_meters()[0] # The aggregated meter that provides the input
train_redd_meter = train_redd_elec.submeters()['microwave'] # The microwave meter that is used as a training target

train_greend_mains = train_greend_elec.mains().all_meters()[0] # The aggregated meter that provides the input
train_greend_meter = train_greend_elec.submeters()['microwave'] # The microwave meter that is used as a training target

print(train_greend_mains)

train_mains = train_redd_mains + train_greend_mains # The aggregated meter that provides the input
train_meter = train_redd_meter + train_redd_meter # The microwave meter that is used as a training target

train_mains = train_elec.mains().all_meters()[0] # The aggregated meter that provides the input
train_meter = train_elec.submeters()['microwave'] # The microwave meter that is used as a training target

gru.train(train_mains, train_meter, epochs=5, sample_period=1)
gru.export_model("model-redd5.h5")

# Test

In [ ]:
# test = DataSet('/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/ukdale/ukdale.hdf5')
# test.set_window(start="30-4-2011")
# test_elec = test.buildings[1].elec
# test_mains = test_elec.mains().all_meters()[0]

test = DataSet('/Users/dariusghomashchian/Desktop/NILM/nilmtk/data/REDD/redd.hdf5')
test.set_window(start="24-10-2014")
test_elec = test.buildings[1].elec
test_mains = test_elec.mains().all_meters()[0]

disag_filename = 'disag-out.h5' # The filename of the resulting datastore
from nilmtk.datastore import HDFDataStore
output = HDFDataStore(disag_filename, 'w')
# test_mains: The aggregated signal meter
# output: The output datastore
# train_meter: This is used in order to copy the metadata of the train meter into the datastore
gru.disaggregate(test_mains, output, train_meter, sample_period=1)
output.store.close()
output.close()



In [ ]:
output.close()

# Results

In [ ]:
result = DataSet(disag_filename)
res_elec = result.buildings[1].elec
predicted = res_elec['microwave']
ground_truth = test_elec['microwave']

import matplotlib.pyplot as plt
predicted.plot(label="kettle")
ground_truth.plot(label="kettle")
plt.show()

In [ ]:
from nilmtk import pmetrics
rpaf = pmetrics.recall_precision_accuracy_f1(predicted, ground_truth)
print("kettle")
print("============ Recall: {}".format(rpaf[0]))
print("============ Precision: {}".format(rpaf[1]))
print("============ Accuracy: {}".format(rpaf[2]))
print("============ F1 Score: {}".format(rpaf[3]))

print("============ Relative error in total energy: {}".format(pmetrics.relative_error_total_energy(predicted, ground_truth)))
print("============ Mean absolute error(in Watts): {}".format(pmetrics.mean_absolute_error(predicted, ground_truth)))